In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sys,os
sys.path.insert(0, '/home/noah/Desktop/large_analysis/ca_analysis')
import fileread as fr
from scipy.signal import fftconvolve
import predefined as pd
import wave_ops as wo
from scipy import signal
import time

def wave(t,*pars):
    t0,tau1,tau2=pars
    return np.heaviside(t-t0,1.)*(np.exp(-(t-t0).astype(float)/tau1)-np.exp(-(t-t0).astype(float)/tau2))

def pixel_cut(array,board,channel):
    return np.logical_and(array['board']==board,array['channel']==channel)

#def precuts(x):
#    x=x[x['energy']>100]
#    x.sort(order='timestamp')
#    t0,t1,t2=x['timestamp'][0:-2],x['timestamp'][1:-1],x['timestamp'][2:]
#    trutharray=land(t2-t1>250,t1-t0>250)
#    x=x[1:-1][trutharray]
#    x=x[lor(x['pileup']<2,x['pilediff']<240)]
#    x=x[x['t0']>100]
#    return x

def linearCombine(a1,b1,offset,c,t0,tau,rise):# (N,*pars):
    N=3500
    t=np.arange(N,dtype=float)
    v=np.zeros((4,N),dtype=float) #np.zeros((len(pars),N))
    t=np.arange(N,dtype=float)
    w=2*np.pi/3500.
    v[0,0:N]=a1*np.sin(w*t)
    v[1,0:N]=b1*np.cos(w*t)
    v[2,0:N]=c*wave(t,t0,tau,rise)
    v[3,0:N]=offset*np.ones(N,dtype=float)
    return np.sum(v,axis=0)

means=np.array([1000, 1031.3367, 1086.8575, 1217.0291, 1041.5563, 1000, 1230.2096, 1188.8999,\
                1000, 1263.1642, 1233.1743, 1056.3289, 1213.4717, 1112.0769, 1049.4534, 1219.0482,\
                1000, 1000, 1077.4932, 1157.1627, 1000, 1163.2235, 1000, 1000,\
                1000, 1027.103, 1111.1212, 1033.5468, 1109.469, 1022.693, 1929.7336, 1000,\
                1000, 1124.478, 1073.1306, 1040.2197, 1100.4457, 1045.0566, 1135.8975, 1073.1854,\
                1000, 1000, 1087.187, 1133.1069, 1005.3494, 1000, 1000, 1000])

def osc_removal(data,output,f):
    '''Data should have baseline restored. NOTE: This removes oscillation AND returns amplitudes '''
    rise=20
    length=len(data['wave'][0])
    omega=2*np.pi*f
    t=np.arange(length)
    DesignT=np.array([np.sin(omega*t),\
                      np.cos(omega*t),\
                      np.ones_like(t),\
                      np.zeros_like(t)])
    shorttraps=np.zeros((48,length))
    wo.multi_trap(arr=shorttraps,rise=20,top=0)
    out=np.zeros(length)
    numpars=DesignT.shape[0]
    yhat=np.zeros(length)
    s2=0
    if len(output) < len(data) or output.shape[1]<2*numpars+1:
        print('Amplitude array should be larger than data array and have atleast '\
            +str(2*numpars+1)+' columns.')
        return

    for i in range(len(data)):
        bdch=data['board'][i]*8+data['channel'][i]
        out =signal.fftconvolve(data['wave'][i],shorttraps[bdch], \
                                'full')[0:length]/float(rise*means[bdch])
        loc = np.argmax(out)-rise
        
        DesignT[numpars-1,0:length]= wave(t,loc,means[bdch],6)
        output[i,0:numpars]=np.matmul(np.matmul(np.linalg.inv(np.matmul(DesignT,DesignT.T)),\
                            DesignT),data['wave'][i])
        yhat=np.matmul(DesignT.T,output[i,0:numpars])
        yhat=data['wave'][i,0:length]-yhat
        s2=np.matmul(yhat.T,yhat)/(length-DesignT.shape[0])
        output[i,numpars:2*numpars]=np.sqrt(s2*np.diag(np.linalg.inv(np.matmul(DesignT,DesignT.T))))
        output[i,2*numpars]=np.sqrt(s2)

        data['wave'][i,0:length]-=(output[i,0]*np.sin(omega*t)\
                                  +output[i,1]*np.cos(omega*t)\
                                  +output[i,2]*np.ones_like(t)).astype('int16')

def osc_removal_double(data,output,f1,f2):
    '''Data should have baseline restored. NOTE: This removes oscillation AND returns amplitudes '''
    rise=20
    length=len(data['wave'][0])
    omega=2*np.pi*f1
    omega2=2*np.pi*f2
    t=np.arange(length)
    DesignT=np.array([np.sin(omega*t),\
                      np.cos(omega*t),\
                      np.sin(omega2*t),\
                      np.cos(omega2*t),\
                      np.ones_like(t),\
                      np.zeros_like(t)])
    shorttraps=np.zeros((48,length))
    wo.multi_trap(arr=shorttraps,rise=20,top=0)
    out=np.zeros(length)
    numpars=DesignT.shape[0]
    yhat=np.zeros(length)
    s2=0
    if len(output) < len(data) or output.shape[1]<2*numpars+1:
        print('Amplitude array should be larger than data array and have atleast '\
            +str(2*numpars+1)+' columns.')
        return

    for i in range(len(data)):
        bdch=data['board'][i]*8+data['channel'][i]
        out =signal.fftconvolve(data['wave'][i],shorttraps[bdch], \
                                'full')[0:length]/float(rise*means[bdch])
        loc = np.argmax(out)-rise
        
        DesignT[numpars-1,0:length]= wave(t,loc,means[bdch],6)
        output[i,0:numpars]=np.matmul(np.matmul(np.linalg.inv(np.matmul(DesignT,DesignT.T)),\
                            DesignT),data['wave'][i])
        yhat=np.matmul(DesignT.T,output[i,0:numpars])
        yhat=data['wave'][i,0:length]-yhat
        s2=np.matmul(yhat.T,yhat)/(length-DesignT.shape[0])
        output[i,numpars:2*numpars]=np.sqrt(s2*np.diag(np.linalg.inv(np.matmul(DesignT,DesignT.T))))
        output[i,2*numpars]=np.sqrt(s2)

        data['wave'][i,0:length]-=(output[i,0]*np.sin(omega*t)\
                                  +output[i,1]*np.cos(omega*t)\
                                  +output[i,3]*np.sin(omega2*t)\
                                  +output[i,4]*np.cos(omega2*t)\
                                  +output[i,5]*np.ones_like(t)).astype('int16')

def osc_removal_triple(data,output,f1,f2,f3):
    '''Data should have baseline restored. NOTE: This removes oscillation AND returns amplitudes '''
    rise=20
    length=len(data['wave'][0])
    omega=2*np.pi*f1
    omega2=2*np.pi*f2
    omega3=2*np.pi*f3
    t=np.arange(length)
    DesignT=np.array([np.sin(omega*t),\
                      np.cos(omega*t),\
                      np.sin(omega2*t),\
                      np.cos(omega2*t),\
                      np.sin(omega3*t),\
                      np.cos(omega3*t),\
                      np.ones_like(t),\
                      np.zeros_like(t)])
    shorttraps=np.zeros((48,length))
    wo.multi_trap(arr=shorttraps,rise=20,top=0)
    out=np.zeros(length)
    numpars=DesignT.shape[0]
    yhat=np.zeros(length)
    s2=0
    if len(output) < len(data) or output.shape[1]<2*numpars+1:
        print('Amplitude array should be larger than data array and have atleast '\
            +str(2*numpars+1)+' columns.')
        return

    for i in range(len(data)):
        bdch=data['board'][i]*8+data['channel'][i]
        out =signal.fftconvolve(data['wave'][i],shorttraps[bdch], \
                                'full')[0:length]/float(rise*means[bdch])
        loc = np.argmax(out)-rise
        
        DesignT[numpars-1,0:length]= wave(t,loc,means[bdch],6)
        output[i,0:numpars]=np.matmul(np.matmul(np.linalg.inv(np.matmul(DesignT,DesignT.T)),\
                            DesignT),data['wave'][i])
        yhat=np.matmul(DesignT.T,output[i,0:numpars])
        yhat=data['wave'][i,0:length]-yhat
        s2=np.matmul(yhat.T,yhat)/(length-DesignT.shape[0])
        output[i,numpars:2*numpars]=np.sqrt(s2*np.diag(np.linalg.inv(np.matmul(DesignT,DesignT.T))))
        output[i,2*numpars]=np.sqrt(s2)

        data['wave'][i,0:length]-=(output[i,0]*np.sin(omega*t)\
                                  +output[i,1]*np.cos(omega*t)\
                                  +output[i,3]*np.sin(omega2*t)\
                                  +output[i,4]*np.cos(omega2*t)\
                                  +output[i,5]*np.sin(omega3*t)\
                                  +output[i,6]*np.cos(omega3*t)\
                                  +output[i,7]*np.ones_like(t)).astype('int16')

        

run='/home/noah/Desktop/large_analysis/ca_analysis/Run_131_0.bin';name=run[-13:-6]
#run='/home/noah/Desktop/large_analysis/ca_analysis/oscillation/Run_0_0.bin';name=run[-11:-6];means=np.ones(48,dtype='float')*1250
numwaves=int((os.stat(run).st_size-8)/7033)
#numwaves=1000
row=0
length=3500
trap = np.zeros((48,length))
rise,top,fall=400,70,1100        #not going to use fall
wo.pixel_traps(workarr=trap,rise=rise,top=top)
traps=np.zeros((numwaves,length))

In [2]:
i=-1
data=fr.raw(run,length=3500,row=row,numwaves=numwaves-row)
wo.baseline_restore(data,600)
pars1=np.zeros((len(data),2*4+1+2))
beg=time.time()
wo.apply_trap(rise=rise,data=data,trap=trap,output=traps)
wo.trap_energy(traps[0:numwaves],length=length,output=pars1[0:,-2]) 
osc_removal(data,pars1,1/3500)
wo.apply_trap(rise=rise,data=data,trap=trap,output=traps)
wo.trap_energy(traps[0:numwaves],length=length,output=pars1[0:,-1])
end=time.time()
print(pars1[i],'\n',end-beg)
np.save('pars1.npy',pars1)


KeyboardInterrupt: 

In [ ]:
data=fr.raw(run,length=3500,row=row,numwaves=numwaves-row)
wo.baseline_restore(data,600)
pars2=np.zeros((len(data),2*6+1+2))
beg=time.time()
wo.apply_trap(rise=rise,data=data,trap=trap,output=traps)
wo.trap_energy(traps[0:numwaves],length=length,output=pars2[0:,-2]) 
osc_removal_double(data,pars2,1/3500,2/3500)
wo.apply_trap(rise=rise,data=data,trap=trap,output=traps)
wo.trap_energy(traps[0:numwaves],length=length,output=pars2[0:,-1])
end=time.time()
print(pars2[i],'\n',end-beg)
np.save('pars2.npy',pars2)

In [ ]:
data=fr.raw(run,length=3500,row=row,numwaves=numwaves-row)
wo.baseline_restore(data,600)
pars3=np.zeros((len(data),2*8+1+2))
beg=time.time()
wo.apply_trap(rise=rise,data=data,trap=trap,output=traps)
wo.trap_energy(traps[0:numwaves],length=length,output=pars3[0:,-2]) 
osc_removal_triple(data,pars3,1/3500,2/35000,.00048)
wo.apply_trap(rise=rise,data=data,trap=trap,output=traps)
wo.trap_energy(traps[0:numwaves],length=length,output=pars3[0:,-1])
end=time.time()
print(pars3[i],'\n',end-beg)
np.save('pars3.npy',pars3)